In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/M1_S2/RITAL/projet2
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install python-terrier
import pandas as pd

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [25]:
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


def prepretraiter(query, ponctuation=True, stem=True, lemme=False, stopword=True):
    """
    Préprétraite une query
    """
    #wn_lemmatizer = nltk.stem.WordNetLemmatizer()
    stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    stopwords_en = stopwords.words('english')
    punc = string.punctuation+'\n\r\t'
    q_traite = []
    for q in query:
        q = q[0].lower()
        if ponctuation:
          q = q.translate(str.maketrans(punc, ' '*len(punc)))
        if stopword:
          q = word_tokenize(q)
          q = ' '.join([w for w in q if w not in stopwords_en])
        #q = ' '.join(map(lambda x: wn_lemmatizer.lemmatize(x), word_tokenize(q)))
        if lemme:
          q = ' '.join(map(lambda x: wordnet_lemmatizer.lemmatize(x), word_tokenize(q)))
        if stem:
          q = ' '.join(map(lambda x: stemmer.stem(x), word_tokenize(q)))
        q_traite.append(q)
    return q_traite

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
def applique_pretraite_passage (df, ponctuation=True, stem=True,lemme=False, stopword=True) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['text']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword)
    df['text']=liste_query
    return df

In [29]:
def applique_pretraite_query (df, ponctuation=True, stem=True,lemme=False, stopword=True) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['query']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword)
    df['query']=liste_query
    df = df.astype({"qid": str, "query": object})
    
    return df

In [22]:
# chargement du json en dataframe
with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
data={"DocumentID" : [int(k0) for k0,v0 in data_row.items() for k1,v1 in v0.items()],
    "docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
      "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
docs_df = pd.DataFrame(data)

In [30]:
# nous faisons les preprocessing
docs_df = applique_pretraite_passage (docs_df, ponctuation=True, stem=False,lemme=True, stopword=True)
docs_df

,DocumentID,docno,text
0,344,"344,1",townspeople lived chartered towns burghers opp...
1,344,"344,0",municipal corporation legal term local governi...
2,344,"344,3",standard varies state state according laws pas...
3,344,"344,2",municipal corporations must also abide organic...
4,344,"344,5",panchayati raj system interacts directly state...
...,...,...,...
50607,479,"479,18",dialects also divided two main chronological c...
50608,479,"479,31",official language moscow novgorod later growin...
50609,479,"479,30",soon followed adoption christianity 988 introd...
50610,479,"479,33",knam va kosk z stu tcit political upheavals ea...


In [17]:
#!rm -drf passageQA

In [31]:
import json
pt_index_path = './passageQA'
if not os.path.exists(pt_index_path + "/data.properties") :
    # création de l'index
    # arg : stemmer=TerrierStemmer.porter, stopwords=TerrierStopwords.terrier, tokeniser=TerrierTokeniser.english
    indexer = pt.DFIndexer(pt_index_path, overwrite=True,stemmer=None,stopwords=None)
    index_ref = indexer.index(docs_df["text"], docs_df["docno"])

else:
    # dans le cas où l'index existe déjà
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

/usr/local/lib/python3.10/dist-packages/pyterrier/index.py:628: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, value in meta_column[1].iteritems():


In [32]:
print(index.getCollectionStatistics().toString())

Number of documents: 50612
Number of terms: 132677
Number of postings: 3385437
Number of fields: 1
Number of tokens: 4139789
Field names: [abstract]
Positions:   false



QUERY

In [35]:
df_train = pd.read_csv("WikiPassageQA/train.tsv",sep='\t')
df_test = pd.read_csv("WikiPassageQA/test.tsv",sep='\t')
df_dev = pd.read_csv("WikiPassageQA/dev.tsv",sep='\t')
#Question None
df_train = df_train[df_train['Question'] != '{}']
df_test = df_test[df_test['Question'] != '{}']
df_dev = df_dev[df_dev['Question'] != '{}']

In [36]:
queries_train=df_train[["QID", "Question"]]
queries_train.columns = ['qid', 'query']
queries_test=df_test[["QID", "Question"]]
queries_test.columns = ['qid', 'query']
queries_dev=df_dev[["QID", "Question"]]
queries_dev.columns = ['qid', 'query']
df_query = pd.concat([df_train, df_test, df_dev])
df_complete = df_query.merge(docs_df, on =["DocumentID"])
df_complete

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",claimed major credit election fernando collor ...
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",evangelicalism major force anglican church 180...
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",rich poor remained traditional catholics evang...
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49",evangelical alliance formed 1846 first ecumeni...
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24",1737 whitefield become national celebrity engl...
...,...,...,...,...,...,...,...
244014,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,72",budgetary rule spend 4 fund year august 2014 g...
244015,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,71",norway fifth largest oil exporter third larges...
244016,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,70",2011 28 state revenues generated petroleum ind...
244017,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,79",two prominent e6 going north south entire coun...


Q_REL

In [37]:
relevant = []
for row in df_complete.iterrows():
  passage = row[1]['RelevantPassages'].split(",")
  doc_passage = row[1]['docno'].split(",")[1] # doc,pass -> [1]
  if doc_passage in passage:
    relevant.append(1)
  else:
    relevant.append(0)

df_complete["label"] = relevant
qrel = df_complete[["QID", "docno","label"]]
qrel.columns = ['qid', 'docno', 'label']
qrel = qrel.astype({"qid": str, "docno": object, "label":object}) #qid en string pour pt.Experiment
qrel

,qid,docno,label
0,3086,"672,42",0
1,3086,"672,48",0
2,3086,"672,43",0
3,3086,"672,49",0
4,3086,"672,24",0
...,...,...,...
244014,2570,"553,72",0
244015,2570,"553,71",0
244016,2570,"553,70",0
244017,2570,"553,79",0


Enseingner sur les info

In [ ]:
# index sur un doc
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 0 #docids are 0-based
#NB: postings will be null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print("%s with frequency %d" % (lee.getKey(),posting.getFrequency()))

In [ ]:
# Index inversé sur un terme
meta = index.getMetaIndex()
inv = index.getInvertedIndex()

le = lex.getLexiconEntry( "montreal" )
# the lexicon entry is also our pointer to access the inverted index posting list
for posting in inv.getPostings( le ):
    docno = meta.getItem("docno", posting.getId())
    print("%s with frequency %d " % (docno, posting.getFrequency()))

In [39]:
stemme = False
lemme=True
stpword = True
queries_train=applique_pretraite_query(queries_train,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)
queries_test=applique_pretraite_query(queries_test,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)
queries_dev=applique_pretraite_query(queries_dev,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)

queries_test

<ipython-input-29-68f805a0f1e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-29-68f805a0f1e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-29-68f805a0f1e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,qid,query
0,449,iraq role political unstabilization
1,1140,warmest years global warming
2,782,olympic games broadcasted
3,1421,election turnout 2008
4,3738,secularism misinterpreted america
...,...,...
411,2522,church mean fallen nature
412,2851,history eurasia
413,3705,nicknames missouri
414,799,endoglossic exoglossic languages coexist nigeria


Les Query Expansion

In [138]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dqe = pt.rewrite.QueryExpansion(index)
kld = pt.rewrite.KLQueryExpansion(index)
rm3 = pt.rewrite.RM3(index)


In [45]:
wc = pt.BatchRetrieve(index, wmodel="Tf")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
ql = pt.BatchRetrieve(index, wmodel="DirichletLM")


pt.Experiment(
    [wc, tf_idf, bm25, ql],
    queries_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names = ["WC","VSM", "BM25", "QL"]
)

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,WC,0.135508,0.164266,0.049038,0.039423,0.290527,0.176482,0.277644,0.384696
1,VSM,0.392988,0.478508,0.144231,0.083413,0.527414,0.489263,0.551242,0.650200
2,BM25,0.391472,0.476182,0.143750,0.083654,0.525963,0.485657,0.550841,0.642188
3,QL,0.358962,0.443990,0.125000,0.076442,0.493455,0.429888,0.512220,0.596835


In [42]:

tf_idf_unnor = lambda keyFreq, posting, entryStats, collStats: posting.getFrequency()*np.log(((1+collStats.getNumberOfDocuments())/(1+entryStats.getDocumentFrequency())))
wc_idf = pt.BatchRetrieve(index, wmodel=tf_idf_unnor)



pt.Experiment(
    [wc_idf],
    queries_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names = ["WC.IDF"]
)

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,WC.IDF,0.176349,0.211362,0.067788,0.053365,0.337116,0.236979,0.375801,0.496074
